We start by importing the necessesary libraries:

In [3]:
import math

from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.chem import MolecularData
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from pyscf import gto, scf, mcscf

from helper_functions import *
from XBK_method import *
from QCC_method import *

import time
import statistics as st
from pprint import pprint

import sys
#!{sys.executable} -m pip install seaborn
#!{sys.executable} -m pip install dwave-ocean-sdk
#!dwave setup


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
cmap = sns.cm.rocket_r
plt.rcParams["figure.figsize"] = (20,3)

We create our molecule and define its properties, along with our chosen basis set:

In [4]:
#create molecule
name = 'HCONH2'
charge = 0
multiplicity = 1
basis = 'sto-6g'

bond_length = 1.1
geometry = get_molGeometry(name, bond_length)
    
molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

Calculating the energies classically:

In [5]:
#run RHF calculations

start_time = time.time()
molecule = run_pyscf(molecule, run_scf=True)
end_time = time.time() - start_time

hf_energy = float(molecule.hf_energy)
hf_data = molecule._pyscf_data['scf']

print(hf_energy)
print(end_time)

-205.53387517069766
1.3430852890014648


In [6]:
#define active space
n_active_electrons = 2
n_active_orbitals = 3
occupied_indices, active_indices = get_active_space(molecule, n_active_electrons, n_active_orbitals)

#run CASCI calculations
casci_data = hf_data.CASCI(n_active_orbitals, n_active_electrons).run(verbose=False)
casci_energy = float(casci_data.e_tot)

print(casci_energy)

-205.54647301011357


Convert from Fermionic basis to qubit and then to Ising model:

In [7]:
#%%capture
#convert to fermionic Hamiltonian
molecular_H = molecule.get_molecular_hamiltonian(occupied_indices=occupied_indices, active_indices=active_indices)
if molecular_H[()] == None:
    molecular_H[()] = 0
fermionic_H = get_fermion_operator(molecular_H)

#add penalty term to ensure correct number of electrons in ground state
weight = 5
penalty_term = FermionOperator('', n_active_electrons)

for i in range(molecular_H.n_qubits):
    penalty_term += FermionOperator(str(i)+'^ '+str(i), -1)
fermionic_H += weight*penalty_term**2

f_operators = list(fermionic_H.get_operators())
#print(len(f_operators))
#print(fermionic_H)

In [8]:
#%%capture
#convert to Pauli operator Hamiltonian
binary_code = bravyi_kitaev_code(molecular_H.n_qubits)
qubit_H = binary_code_transform(fermionic_H, binary_code)
qubit_H.compress()

#apply symmetry reductions and calculate minimum eigenvalue (should be equal to CASCI energy)
sectors = taper_qubits(qubit_H)
qubit_H, min_eigenvalue = sector_with_ground(sectors)
m = count_qubits(qubit_H)

print(min_eigenvalue, '\n')
operators = list(qubit_H.get_operators())
#print(len(operators))
#print(m)

-205.5464730101142 



Connect to DWave machine:

In [9]:
#set sampler to perform the annealing
#from neal import SimulatedAnnealingSampler
#sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

## Real Device Backend
from dwave.system.samplers import DWaveSampler
sampler = DWaveSampler()

print("Connected to sampler", sampler.solver.name)
sampler = EmbeddingComposite(sampler)


Connected to sampler DW_2000Q_6


Define functions for running the QCC algorithm, for testing different parameters, and for plotting:

In [10]:
### QCC method ###

def QCC_m(angle_folds=3, amplitude_folds=1, num_cycles=10, num_samples=1000, strength=1e3):
    #set number of Bloch angle and entangler amplitude foldings
    #angle_folds = 3
    #amplitude_folds = 1
#    print("Angle folds: ", angle_folds, "\nAmplitude Folds: ", amplitude_folds, 
#          "\nNum cycles: ", num_cycles, "\nStrength: ", strength)

    #create dictionary of QubitOperator entanglers
    entanglers = {'IYZI': QubitOperator('Y1 Z2'), 'IZYI': QubitOperator('Z1 Y2'),
                  'IXYI': QubitOperator('X1 Y2'), 'IYXI': QubitOperator('Y1 X2')}

    #run QCC method
    sampler = DWaveSampler()
    sampler = EmbeddingComposite(sampler)

    QCC_energy, variables, response = QCC(qubit_H, entanglers, angle_folds, amplitude_folds, sampler,
                                num_cycles=num_cycles, num_samples=num_samples, strength=strength)
    
    #print(response.info["timing"])
    return QCC_energy, response.info["timing"]

#print("Finished.")
#print(QCC_energy)
#print(variables)

In [11]:
### Test method ###

def Test(QPU=False, num_cycles=[3,4,5,6,7], num_samples=[100,300,500,700,900], strengths=[1000], runs=5, angle_folds=3, amplitude_folds=1):
    results = []
    for cycle in num_cycles:
        for sample in num_samples:
            for strength in strengths:
                this_time = []
                this_energy = []
                if QPU:
                    for i in range(0,runs):
                        response = QCC_m(3, 1, cycle, sample, strength)
                        energy = response[0]
                        print("Energy: ", energy)
                        this_energy.append(energy - hf_energy)
                        this_time.append(response[1]['qpu_access_time'])
                        print("Time: ", response[1]['qpu_access_time'])
                        print("Run ", i, "completed.")
                    results.append([cycle, sample, strength, st.mean(this_energy), st.stdev(this_energy), st.mean(this_time), st.stdev(this_time)])
                    print("Result ", cycle, sample, strength, " finished.")
                else:
                    for i in range(0,runs):
                        start_time = time.time()
                        energy = QCC_m(3, 1, cycle, sample, strength)
                        print(energy)
                        end_time = time.time() - start_time
                        this_energy.append(energy - hf_energy)
                        this_time.append(end_time)
                        print("Run ", i, "completed.")
                    results.append([cycle, sample, strength, st.mean(this_energy), st.stdev(this_energy), st.mean(this_time), st.stdev(this_time)])
                    print("Result ", cycle, sample, strength, " finished.")
    return results

In [12]:
def plot_heatmap(results, xvar="num_samples", yvar="num_cycles", const="str"):
    titles=["Average energy error for number of samples vs number of cycles",
           "Standard deviation in energy error for number of samples vs number of cycles",
           "Average time for number of samples vs number of cycles"]

    for i in range(3,6):
        data = list(chunks(([result[i] for result in results[0::tot_str]]),tot_samples))

        vmax = max_value(data)
        if(i!=5):
            vmax/=10
        vmin = min_value(data)

        ax = sns.heatmap(data, vmin=vmin, vmax=vmax, xticklabels=num_samples, yticklabels=num_cycles, cmap=cmap)
        plt.xlabel("Number of samples")
        plt.ylabel("Number of cycles")
        plt.title(titles[i-3])
        plt.show()

Running a small demo over just 4 parameters, 3 runs each, 12 runs total.

We see that in many cases, the energy blows up (expected value around -205). Also occassionally the error "ValueError: no embedding found" appears. 

In [16]:
num_cycles=[4,5]
num_samples=[250,500]
strengths=[1000]
runs=3
tot_cycles=len(num_cycles)
tot_samples=len(num_samples)
tot_str=len(strengths)
print(tot_cycles)
print(tot_samples)

results_QPUTest = Test(True, num_cycles, num_samples, strengths, runs)
pprint(results_QPUTest)

2
2
Energy:  12811.899075119582
Time:  71041.1
Run  0 completed.
Energy:  -205.4520072061423
Time:  70664.1
Run  1 completed.
Energy:  1794.7508064952563
Time:  70762.3
Run  2 completed.
Result  4 250 1000  finished.
Energy:  -205.53763206052827
Time:  130442.7
Run  0 completed.
Energy:  10824.044112685602
Time:  130708.3
Run  1 completed.
Energy:  794.5479927947745
Time:  130550.9
Run  2 completed.
Result  4 500 1000  finished.


ValueError: no embedding found

Let's try one more run today (6/20/22)

In [13]:
num_cycles=[5]
num_samples=[250,500]
strengths=[100,1000]
runs=3
tot_cycles=len(num_cycles)
tot_samples=len(num_samples)
tot_str=len(strengths)
print(tot_cycles)
print(tot_samples)

results_QPUTest = Test(True, num_cycles, num_samples, strengths, runs)
pprint(results_QPUTest)

1
2
Energy:  -205.4979681592058
Time:  70606.1
Run  0 completed.


ValueError: no embedding found

Here only one run completed before embedding error occurred.

Below code is old tests, in both tests no errors but occasional energy blowup.

In [13]:
num_cycles=[4]
num_samples=[250]
strengths=[1000]
runs=5

results_QPUTest = Test(True, num_cycles, num_samples, strengths, runs)
pprint(results_QPUTest)

Energy:  794.5741843677824
Time:  70898.3
Run  0 completed.
Energy:  -205.14294722753402
Time:  70884.7
Run  1 completed.
Energy:  789.4239689143142
Time:  70884.5
Run  2 completed.
Energy:  805.0207053476479
Time:  70884.3
Run  3 completed.
Energy:  10830.837969870132
Time:  70951.9
Run  4 completed.
Result  4 250 1000  finished.
[[4,
  250,
  1000,
  2808.4766514251664,
  4619.93441374712,
  70900.74,
  29.217255175663695]]


In [14]:
num_cycles=[4]
num_samples=[250]
strengths=[1000]
runs=5

results_QPUTest = Test(True, num_cycles, num_samples, strengths, runs)
pprint(results_QPUTest)

Energy:  13804.234388596728
Time:  70962.3
Run  0 completed.
Energy:  -205.53693288384238
Time:  70950.7
Run  1 completed.
Energy:  -205.53791769311647
Time:  70791.1
Run  2 completed.
Energy:  2815.5632122055104
Time:  70794.7
Run  3 completed.
Energy:  -205.53387517069496
Time:  70591.7
Run  4 completed.
Result  4 250 1000  finished.
[[4,
  250,
  1000,
  3406.171650181615,
  6070.226681585684,
  70818.1,
  150.75635973318109]]


In [30]:
#QPU
test_result = QCC_m(3, 1, 5, 500, 1000)
print(test_result)

-205.54390640774363


In [14]:
#QPU
test_result = QCC_m(3, 1, 5, 500, 1000)
print(test_result)

-205.5439064077509


In [27]:
test_result1 = QCC_m(3, 1, 4, 500, 1000)
print("Test 1: Energy: ", test_result1[0], "\nTiming info: \n", test_result1[1])

Test 1: Energy:  -205.54390640795464  Timing info: 
 {'qpu_sampling_time': 119470.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 198.4, 'qpu_access_time': 130285.7, 'qpu_access_overhead_time': 1457.3, 'qpu_programming_time': 10815.7, 'qpu_delay_time_per_sample': 20.54, 'post_processing_overhead_time': 773.0, 'total_post_processing_time': 773.0}


In [41]:
start_time = time.time()
test_result2 = QCC_m(3, 1, 5, 500, 1000)
end_time2 = time.time() - start_time
print("Test 2: Energy: ", test_result2, " Time: ", end_time2)

Test 2: Energy:  -205.54647300940997  Time:  24.456588983535767


In [40]:
start_time = time.time()
test_result3 = QCC_m(3, 1, 4, 250, 1000)
end_time3 = time.time() - start_time
print("Test 3: Energy: ", test_result3, " Time: ", end_time3)

Test 3: Energy:  -202.6327709092584  Time:  138.12823510169983


In [42]:
start_time = time.time()
test_result4 = QCC_m(3, 1, 5, 250, 1000)
end_time4 = time.time() - start_time
print("Test 4: Energy: ", test_result4, " Time: ", end_time4)

Test 4: Energy:  7849.936954598757  Time:  77.70472645759583


In [43]:
start_time = time.time()
test_result5 = QCC_m(3, 1, 5, 500, 1000)
end_time5 = time.time() - start_time
print("Test 5: Energy: ", test_result5, " Time: ", end_time5)

Test 5: Energy:  -144.50730588356964  Time:  112.0889539718628
